<a href="https://colab.research.google.com/github/anahi-cruz7/Diplomado_PUCP/blob/main/PC4_Causal_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.19.5
!pip install scikit-learn==1.2.2
!pip install econml==0.13.0

  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (numpy)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from econml.cate_interpreter import SingleTreeCateInterpreter
from econml.dml import LinearDML

In [ ]:
np.int = np.int64

In [ ]:
from google.colab import files
import io

uploaded = files.upload()

In [ ]:
print(uploaded.keys())

In [ ]:
ab_data = pd.read_csv(io.BytesIO(uploaded['ab_data.csv']))
countries = pd.read_csv(io.BytesIO(uploaded['countries.csv']))

In [ ]:
ab_data

In [ ]:
# Task 1: Merge de ambas bases de datos basado en la variable user_id
df = pd.merge(ab_data, countries, on='user_id', how='inner')
df

In [ ]:
# Verificar si 'merged_data' tiene valores nulos
df.isnull().values.any()

In [ ]:
df.converted.value_counts()

In [ ]:
df = df[((df['group'] == 'treatment') & (df['landing_page'] == 'new_page')) |
                             ((df['group'] == 'control') & (df['landing_page'] == 'old_page'))]

In [ ]:
df

In [ ]:
# Task 3: Construcción del propensity score
# Convertir el timestamp a formato numérico
df['timestamp'] = df['timestamp'].apply(lambda x: sum(float(t) * 60**i for i, t in enumerate(reversed(x.split(':')))))

In [ ]:
df

In [ ]:
df.country.value_counts()

In [ ]:
country_dummies = pd.get_dummies(df['country'], prefix='country', drop_first=True)
df = pd.concat([df, country_dummies], axis=1)
df

In [ ]:
X = ['timestamp'] + list(country_dummies.columns)
X

In [ ]:
df['pscore'] = LogisticRegression().fit(df[X], df['landing_page']).predict_proba(df[X])[:,1]

sns.histplot(data=df, x='pscore', hue='landing_page').set(title='Predicted propensity scores')
plt.show()

In [ ]:
df['landing_page'] = np.where(df['landing_page'] == 'new_page', 1, 0)
df

In [ ]:
df['pscore'] = LogisticRegression().fit(df[X], df['landing_page']).predict_proba(df[X])[:,1]


sns.histplot(data=df, x='pscore', hue='landing_page').set(title='Predicted propensity scores')
plt.show()

In [ ]:
df['y_star'] = df['converted'] / (df['landing_page'] * df['pscore'] - (1-df['landing_page']) * (1-df['pscore']))

In [ ]:
sns.histplot(data=df, x='y_star', binwidth= 8)
plt.axvline(x=df['y_star'].mean(), color='red', linestyle='--')